In [179]:
import os.path
import bqplot
from IPython.display import display, HTML
import ipywidgets
import numpy as np
import pandas as pd

solutions = pd.read_csv(os.path.join('data', 'overview', 'solutions.csv'),
                                    index_col=False, skipinitialspace=True, header=0,
                                    skip_blank_lines=True, comment='#')
soln_results = pd.read_csv(os.path.join('data', 'overview', 'populationdemo.csv'),
                                    index_col=False, skipinitialspace=True, header=0,
                                    skip_blank_lines=True, comment='#')
solutions = solutions.merge(soln_results, on='Solution', how='left')

colormap = {
    'Materials': 'RebeccaPurple',
    'Electricity Generation': 'Peru',
    'Food': 'FireBrick',
    'Land Use': 'Green',
    'Women and Girls': 'DarkGoldenRod',
    'Transport': 'Teal',
    'Buildings and Cities': 'SteelBlue',
}

def get_color(sector):
    return colormap.get(sector, 'Beige')

soln_data = solutions['CO2eq-pop9B'].tolist()
soln_labels = solutions['Solution'].tolist()
soln_colors = solutions['Sector'].apply(get_color).tolist()
soln_tt = bqplot.Tooltip(fields=['label', 'size'], formats=['', '0.1f'], labels=['Name', 'GTons-CO2'])
soln_pie = bqplot.Pie(sizes=soln_data, display_labels='none', labels=soln_labels, sort=True, tooltip=soln_tt,
                 display_values=True, values_format='.1f', radius=170, inner_radius=150,
                 colors=soln_colors, stroke='Gainsboro')
soln_pie.tooltip_style = {'opacity': 0.7, 'background-color': 'White', 'border': 'none', 'border-collapse': 'collapse'}

sectors = solutions.pivot_table(index='Sector',aggfunc=sum)
sctr_data = sectors['CO2eq-pop9B'].tolist()
sctr_labels = sectors.index.tolist()
sctr_colors = [get_color(v) for v in sctr_labels]
sctr_tt = bqplot.Tooltip(fields=['label', 'size'], formats=['', '0.1f'], labels=['Name', 'GTons-CO2eq'])
sctr_pie = bqplot.Pie(sizes=sctr_data, display_labels='none', labels=sctr_labels, sort=True, tooltip=sctr_tt,
                 display_values=True, values_format='.1f', radius=138, inner_radius=118,
                 colors=sctr_colors, stroke='Gainsboro')
sctr_pie.tooltip_style = {'opacity': 0.7, 'background-color': 'White', 'border': 'none', 'border-collapse': 'collapse'}
solution_chart = bqplot.Figure(marks=[soln_pie, sctr_pie], animation_duration=1000)
solution_chart.background_style = {'fill': 'White'}

In [189]:
solutions['CO2eq'] = solutions['CO2eq-pop9B']
solution_list = ipywidgets.HTML(solutions.loc[:, ['Solution', 'Sector', 'CO2eq']].to_html())

In [195]:
pop = ipywidgets.FloatSlider(
    value=9.0,
    min=8.0,
    max=9.0,
    step=0.01,
    description='Pop (B):',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

def update_solutions(population):
    delta = (solutions['CO2eq-pop9B'] - solutions['CO2eq-pop8B']) * (population - 8.0)
    solutions['CO2eq'] = solutions['CO2eq-pop8B'] + delta
    soln_pie.sizes = solutions['CO2eq'].tolist()
    sectors = solutions.pivot_table(index='Sector',aggfunc=sum)
    sctr_pie.sizes = sectors['CO2eq'].tolist()
    solution_list.value = solutions.loc[:, ['Solution', 'Sector', 'CO2eq']].to_html()

p = ipywidgets.interactive(update_solutions, population=pop)
display(p)

h = ipywidgets.HBox(children=[solution_list, solution_chart])
h.layout.justify_content = 'space-between'
h

interactive(children=(FloatSlider(value=9.0, continuous_update=False, description='Pop (B):', max=9.0, min=8.0…